In [1]:
# import libararies
# data manupulation
import pandas as pd
import numpy as np
from scipy.stats import norm
# visulization
import cufflinks as cf
cf.set_config_file(offline=True, dimensions=((1000,600)), theme = 'white')
# dataframe display format
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [2]:
# European option

# sepcify parameters for FDM
T = 1                        # time to maturity
E = 100                      # strike price
r = 0.05                     # risk free rate
vol = 0.20                   # stock volatility
flag = 1                     # 1 for call, -1 for put
nas = 20                     # number of asset steps 
# grid
ds = 2*E/nas                 # asset step size
dt = 0.9/((vol**2)*(nas**2)) # time step size, for fourier stability (< 1/(vol**2 * nas**2))
nts = int(T/dt) + 1          # back-out number of time steps
dt = T/nts                   # final time step size

In [3]:
# step 1: generate grid by specifying grid points

# create asset steps i*ds
s = np.arange(0, (nas+1)*ds, ds) # from 0 to 2E with step size as ds
s

array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
       110., 120., 130., 140., 150., 160., 170., 180., 190., 200.])

In [4]:
# create time steps
t = np.arange(0, (nts+1)*dt, dt) # from 0 to T with step size as dt
t

array([0.        , 0.05555556, 0.11111111, 0.16666667, 0.22222222,
       0.27777778, 0.33333333, 0.38888889, 0.44444444, 0.5       ,
       0.55555556, 0.61111111, 0.66666667, 0.72222222, 0.77777778,
       0.83333333, 0.88888889, 0.94444444, 1.        ])

In [5]:
# verify step sizes
s.shape, t.shape

((21,), (19,))

In [6]:
# intialize the grid with zeros
grid = np.zeros((len(s), len(t)))

# convert grid into dataframe: asset steps as index (y) and time steps as columns (x)
grid = pd.DataFrame(grid, index=s, columns=np.around(t,3))
grid

,0.000,0.056,0.111,0.167,0.222,0.278,0.333,0.389,0.444,0.500,0.556,0.611,0.667,0.722,0.778,0.833,0.889,0.944,1.000
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# step 2: specify the final or initial condition at expiration

# the first column as maturity
if flag == 1:   
    # call option
    grid.iloc[:, 0] = np.maximum(s-E, 0) 
else:
    # put option
    grid.iloc[:,0] = np.maximum(E-s, 0) 

grid

,0.000,0.056,0.111,0.167,0.222,0.278,0.333,0.389,0.444,0.500,0.556,0.611,0.667,0.722,0.778,0.833,0.889,0.944,1.000
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# step 3: use boundary conditions to calculate option values and step back down to fill the grid

# k is a counter here, k=t=time-to-maturity
for k in range(1, len(t)): # k loop time steps (nts-1 given skip k=0)
    for i in range(1, len(s)-1): # i loop asset steps (nas-2 given skip s=0 and s=infinity)
        delta = (grid.iloc[i+1, k-1] - grid.iloc[i-1, k-1]) / (2*ds)   # central difference approximation
        gamma = (grid.iloc[i+1, k-1] - 2*grid.iloc[i, k-1] + grid.iloc[i-1, k-1]) / (ds**2)
        theta = (-0.5*vol**2*s[i]**2*gamma) - (r*s[i]*delta) + (r*grid.iloc[i, k-1])
        
        grid.iloc[i, k] = grid.iloc[i, k-1] - (theta*dt)
        
    # set boundary condition at s=0
    # ds = r*s*dt + sigma*s*dx, when s=0, ds=0
    grid.iloc[0, k] = grid.iloc[0, k-1] * (1-r*dt)
        
    # set boundary condition at s=infinity
    # gamma = 0, so you can linearly interpolate or extract the values
    grid.iloc[len(s)-1, k] = 2*grid.iloc[len(s)-2, k] - grid.iloc[len(s)-3, k]
    
    # round result to 3 digits
    grid = np.around(grid, 3)

grid

,0.000,0.056,0.111,0.167,0.222,0.278,0.333,0.389,0.444,0.500,0.556,0.611,0.667,0.722,0.778,0.833,0.889,0.944,1.000
0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
10.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
20.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
30.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
40.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.002,0.003,0.004,0.006
60.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.002,0.004,0.007,0.011,0.016,0.023,0.031,0.041,0.053,0.066
70.0,0.0,0.000,0.000,0.000,0.001,0.003,0.008,0.016,0.028,0.045,0.067,0.094,0.126,0.164,0.207,0.255,0.308,0.366,0.429
80.0,0.0,0.000,0.000,0.011,0.037,0.080,0.141,0.218,0.310,0.416,0.534,0.662,0.799,0.944,1.096,1.254,1.417,1.584,1.755
90.0,0.0,0.000,0.128,0.336,0.592,0.878,1.181,1.494,1.812,2.131,2.450,2.767,3.081,3.392,3.700,4.005,4.306,4.604,4.899


In [9]:
# visualization

# plot call option payoff
grid.iplot(kind='surface', title='call option values by FDM')

In [10]:
# bilinear interpolatation for tenor points not included in grid

def bilinear_interpolation(asset_price, ttm, df):
    
    # find neighbouring rows and column
    col1 = df.columns[df.columns<ttm][-1]
    col2 = df.columns[df.columns>=ttm][0]
    row1 = df.index[df.index<asset_price][-1]
    row2 = df.index[df.index>=asset_price][0]
    
    # define points and areas
    # collect option value V1, V2, V3, V4 into V
    V = [df.loc[row1, col1], df.loc[row1, col2], 
         df.loc[row2, col2], df.loc[row2, col1]]
    # collect area value A1, A2, A3, A4 into A
    A = [(row2-asset_price)*(col2-ttm),
        (row2-asset_price)*(ttm-col1),
        (asset_price-row1)*(ttm-col1),
        (asset_price-row1)*(col2-ttm)
        ]
    
    # interpolate the values
    return sum(np.array(A)*np.array(V))/sum(np.array(A))


# example
bilinear_interpolation(105, 0.3, grid)

8.4765

In [11]:
# convert grid caculation into function

def efdm_grid(Strike, Volatility, Rate, TTM, NAS, Flag=1):
    
    # Specify Flag as 1 for calls and -1 for puts
    
    ds = 2*Strike/NAS                   # asset step size
    dt = 0.9/Volatility**2/NAS**2       # for stability
    
    NTS = int(TTM / dt) + 1             # time step size, alternatively use fixed size 10 on stability issue 
    dt = TTM/NTS                        # time step
    
    s = np.arange(0,(NAS+1)*ds,ds)
    t = TTM-np.arange(NTS*dt,-dt,-dt)
    
    # Initialize the grid with zeros
    grid = np.zeros((len(s),len(t)))
    grid = pd.DataFrame(grid, index=s, columns=np.around(t,2))
    
    # Set boundary condition at Expiration
    grid.iloc[:,0] = abs(np.maximum(Flag * (s - Strike), 0))
        
    for k in range(1, len(t)):
        for i in range(1,len(s)-1):
            delta = (grid.iloc[i+1,k-1] - grid.iloc[i-1,k-1]) / (2*ds)
            gamma = (grid.iloc[i+1,k-1]-2*grid.iloc[i,k-1]+grid.iloc[i-1,k-1]) / (ds**2)
            theta = (-0.5* vol**2 * s[i]**2 * gamma) - (r*s[i]*delta) + (r*grid.iloc[i,k-1])

            grid.iloc[i,k] = grid.iloc[i,k-1] - dt*theta
    
        # Set boundary condition at S = 0
        grid.iloc[0,k] = grid.iloc[0,k-1] * (1-r*dt)

        # Set boundary condition at S = infinity
        grid.iloc[len(s)-1,k] = abs(2*(grid.iloc[len(s)-2,k]) - grid.iloc[len(s)-3,k])
            
    # round grid values to 4 decimals
    return np.around(grid,2)

In [12]:
fdm_puts = efdm_grid(100, 0.2, 0.05, 1, 20, -1)
fdm_puts

,0.00,0.06,0.11,0.17,0.22,0.28,0.33,0.39,0.44,0.50,0.56,0.61,0.67,0.72,0.78,0.83,0.89,0.94,1.00
0.0,100.0,99.72,99.45,99.17,98.89,98.62,98.34,98.07,97.80,97.53,97.26,96.99,96.72,96.45,96.18,95.91,95.65,95.38,95.12
10.0,90.0,89.72,89.45,89.17,88.89,88.62,88.34,88.07,87.80,87.53,87.26,86.99,86.72,86.45,86.18,85.91,85.65,85.38,85.12
20.0,80.0,79.72,79.45,79.17,78.89,78.62,78.34,78.07,77.80,77.53,77.26,76.99,76.72,76.45,76.18,75.91,75.65,75.38,75.12
30.0,70.0,69.72,69.45,69.17,68.89,68.62,68.34,68.07,67.80,67.53,67.26,66.99,66.72,66.45,66.18,65.91,65.65,65.38,65.12
40.0,60.0,59.72,59.45,59.17,58.89,58.62,58.34,58.07,57.80,57.53,57.26,56.99,56.72,56.45,56.18,55.91,55.65,55.38,55.12
50.0,50.0,49.72,49.45,49.17,48.89,48.62,48.34,48.07,47.80,47.53,47.26,46.99,46.72,46.45,46.18,45.92,45.65,45.39,45.12
60.0,40.0,39.72,39.45,39.17,38.89,38.62,38.35,38.07,37.80,37.53,37.26,36.99,36.73,36.46,36.20,35.94,35.69,35.43,35.18
70.0,30.0,29.72,29.45,29.17,28.89,28.62,28.35,28.09,27.83,27.57,27.32,27.08,26.84,26.61,26.39,26.17,25.96,25.75,25.55
80.0,20.0,19.72,19.45,19.18,18.93,18.70,18.49,18.29,18.11,17.94,17.79,17.65,17.52,17.39,17.28,17.17,17.06,16.96,16.87
90.0,10.0,9.72,9.57,9.50,9.49,9.50,9.53,9.57,9.61,9.66,9.71,9.75,9.80,9.84,9.88,9.92,9.95,9.98,10.01


In [13]:
# visulization
fdm_puts.iplot(kind='surface')

In [14]:
# convergence analysis

nas_list = [10, 20, 30, 40, 50, 60]
fdmoption = []

for i in nas_list:
    fdmoption.append(efdm_grid(100, 0.2, 0.05, 1, i).loc[100, 1])

fdmoption

[9.51, 10.26, 10.37, 10.4, 10.42, 10.43]

In [15]:
# rewrite BSM into a class (from last pythonlab)
class BS:
    
    """
    this is a class for options contract for pricing european options on stocks without dividends
    
    attributes:
        spot       : int or float
        strike     : int or float
        rate       : int or float
        ttm        : int or float (days to expiry in number of years)
        vol        : float
    """
    
    def __init__(self, spot, strike, rate, ttm, vol):
        
        # spot price
        self.spot = spot
        
        # strike price
        self.strike = strike
        
        # risk-free rate
        self.rate = rate
        
        # days to expire
        self.ttm = ttm
        
        # stock vol
        self.vol = vol
        
        # calculated utility (d2=d1-utility)
        self._a_ = self.vol * (self.ttm ** 0.5)
        
        # calculated d1
        if self.strike == 0:
            raise ZeroDevisionError('the strike price cannot be zero') # raise an exception
        else:
            self._d1_ = (np.log(self.spot/self.strike) + (self.rate + 0.5 * (self.vol**2))*self.ttm ) / self._a_
        
        # calculated d2
        self._d2_ = self._d1_ - self._a_
        
        # calculated discount factor
        self._b_ = np.exp(-(self.rate * self.ttm))
        
        
        # The __dict__ attribute
        '''
        Contains all the attributes defined for the object itself. It maps the attribute name to its value.
        '''
        for i in ['callPrice', 'putPrice', 'callDelta', 'putDelta', 'callTheta', 'putTheta', \
                  'callRho', 'putRho', 'vega', 'gamma']:
            self.__dict__[i] = None
        
        [self.callPrice, self.putPrice] = self._price()
        [self.callDelta, self.putDelta] = self._delta()
        [self.callTheta, self.putTheta] = self._theta()
        [self.callRho, self.putRho] = self._rho()
        self.vega = self._vega()
        self.gamma = self._gamma()
        
    
    # Option Price
    def _price(self):
        '''Returns the option price: [Call price, Put price]'''

        if self.vol == 0 or self.ttm == 0:
            call = np.maximum(0.0, self.spot - self.strike)
            put = np.maximum(0.0, self.strike - self.spot)
        else:
            call = self.spot * norm.cdf(self._d1_) - self.strike * self._b_ * norm.cdf(self._d2_)
            put = self.strike * self._b_ * norm.cdf(-self._d2_) - self.spot * norm.cdf(-self._d1_)
            
        return [call, put]

    
    # Option Delta
    def _delta(self):
        '''Returns the option delta: [Call delta, Put delta]'''

        if self.vol == 0 or self.ttm == 0:
            call = 1.0 if self.spot > self.strike else 0.0
            put = -1.0 if self.spot < self.strike else 0.0
        else:
            call = norm.cdf(self._d1_)
            put = -norm.cdf(-self._d1_)
        
        return [call, put]

    
    # Option Gamma
    def _gamma(self):
        '''Returns the option gamma'''
        return norm.pdf(self._d1_) / (self.spot * self._a_)

    
    # Option Vega
    def _vega(self):
        '''Returns the option vega'''
        if self.vol == 0 or self.ttm == 0:
            return 0.0
        else:
            return self.spot * norm.pdf(self._d1_) * self.ttm**0.5 / 100

        
    # Option Theta
    def _theta(self):
        '''Returns the option theta: [Call theta, Put theta]'''
        call = -self.spot * norm.pdf(self._d1_) * self.vol / (2 * self.ttm**0.5) - self.rate * self.strike * self._b_ * norm.cdf(self._d2_)
        put = -self.spot * norm.pdf(self._d1_) * self.vol / (2 * self.ttm**0.5) + self.rate * self.strike * self._b_ * norm.cdf(-self._d2_)   
        
        return [call / 365, put / 365]

    
    # Option Rho
    def _rho(self):
        '''Returns the option rho: [Call rho, Put rho]'''
        call = self.strike * self.ttm * self._b_ * norm.cdf(self._d2_) / 100
        put = -self.strike * self.ttm * self._b_ * norm.cdf(-self._d2_) / 100

        return [call, put]

In [16]:
# instantiate the object
option = BS(100,100,0.05,1,0.20)
bsoption = round(option.callPrice,2)
bsoption = bsoption.repeat(len(nas_list))

bsoption

array([10.45, 10.45, 10.45, 10.45, 10.45, 10.45])

In [17]:
# put FDM and BS results into dataframe
df = pd.DataFrame(list(zip(bsoption,fdmoption)), columns=['BS', 'FDM'], index=nas_list)
df['dev'] = df['FDM'] - df['BS']
df['% dev'] = round(df['dev'] / df['BS']*100,2)

# output
print('BS-FDM Convergence over NAS')
df

BS-FDM Convergence over NAS


,BS,FDM,dev,% dev
10,10.45,9.51,-0.94,-9.00
20,10.45,10.26,-0.19,-1.82
30,10.45,10.37,-0.08,-0.77
40,10.45,10.40,-0.05,-0.48
50,10.45,10.42,-0.03,-0.29
60,10.45,10.43,-0.02,-0.19
